**<font size="5">Treino com os dados do Titanic</font>**

Um dos naufrágios mais infames da história é o do Titanic, que afundou após colidir com um iceberg. Infelizmente, não havia botes salva-vidas suficientes para todos a bordo, resultando na morte de 1502 dos 2224 passageiros e tripulantes.

Embora houvesse algum elemento de sorte envolvido na sobrevivência, parece que alguns grupos de pessoas eram mais propensos a sobreviver do que outros. **O desafio aqui é construir um modelo preditivo que discrimine os grupos de pessoas com maiores chances de sovreviver.**

**<font size="3">Principais objetivos</font>**: 
- Me familiarizar com a plataforma Kaggle e suas competições.
- Desenvolver e colocar em prática técnicas de EDA
- Resolver o problema e buscar melhorar a pontuação baseando-se em metodos e ideias que vi em notebooks compartilhados por cientistas de dados da comunidade mais experientes.

**<font size="5">Importando dados e bibliotecas necessárias</font>**

In [ ]:
# análise e transformação dos dados
import pandas as pd
import numpy as np
import random as rnd

# visualização
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# importando os datasets

import os

train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
all_data = train_df.append(test_df, sort=True);

In [ ]:
train_df.head(8)

**<font size="5">Análise exploratória</font>**

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
# Número de "NaN"s em cada feature
print(all_data.shape)
all_data.isnull().sum().drop('Survived')

**<font size="3">Observações</font>**:
- Cinco features são nominais, das quais apenas "Sex" e "Embarked" possuem poucos valores únicos e podem ser facilmente convertidas em variáveis numéricas ordinais/discretas.

- Proporção de "Missing values" encontrados: "Cabin" 77%  >>> "Age" 20% >> "Embarked" apenas 2 > "Fare" apenas 1.

- A chance de sobrvivência média do conjunto de dados é de 38% e está próxima da taxa real de 32% (1-1502/2224). Também pode ser usado como parâmetro de comparação para o impacto de determinadas features.

**<font size="3">Especulações</font>**:
- PassangerId, Ticket e Name provavelmente não contribuem para a sobrevivência, descartá-los logo no início é conveniente pois reduz o volume de dados, acelera o processamento do código e simplifica a análise, mas vale ressaltar que o nome também possui informações sobre o título do passageiro, extraír isso em uma nova feature pode beneficiar a acuracia do modelo.

- Uma possibilidade para haver tantos valores nulos para a cabine é que isso pode representar uma luxúria de alguns poucos passageiros, possivelmente um indicador de sua influência. Transforma-la em uma variável binária pode beneficiar o modelo. 

- É importante saber quais features se correlacionam com a sobrevivência e entre sí logo no início do projeto, pois isso guia a tomada de decisão sobre quais delas manter e transformar; alguns dados como o valor da tarifa (Fare) e classe (Pclass) podem acabar dizendo a mesma coisa ficando redundantes

- Se queremos fazer correlações logo de início, é importante converter features potencialmente relevantes como  "Sex" e "Embarked" para variáveis ordinais/discretas antes.

- Por fim, se a análise apontar que features incompletas como "Age" e "Embarked" impactam na sobrevivência, devem ser completadas ao invés de descartadas.

In [ ]:
# Extraindo os títulos de "Name"
all_data['Title'] = all_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(all_data['Title'], all_data['Sex']).T

In [ ]:
# Transformando a feature "Cabin" em binária
all_data['Cabin'].fillna(0, inplace=True)
all_data.loc[all_data['Cabin'] != 0, 'Cabin'] = 1

In [ ]:
# Descartando dados irrelevantes

Id_test = test_df['PassengerId'] # necessário para submissão do projeto.

all_data = all_data.drop(['PassengerId', 'Ticket', 'Name'], axis=1)

train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):]

**<font size="5">Checando correlações com a sobrevivência</font>**

In [ ]:
def pivota_feature_com_sobrev(feature_analisada):
    
    #Essa função cria um pequeno DataFrame com a taxa de sobrevivência 
    #e o número de indivíduos de cada elemento de uma feature.
    
    df_pivot = pd.concat([train_df[feature_analisada].rename('# ind').value_counts(), 
                          train_df[[feature_analisada, "Survived"]].groupby(feature_analisada, as_index=True).mean()], 
                         axis=1, sort=True)
    
    df_pivot.index.name = feature_analisada
    
    return round(df_pivot, 3)

In [ ]:
display(pivota_feature_com_sobrev('Sex'), 
        pivota_feature_com_sobrev('Embarked'), 
        pivota_feature_com_sobrev('Pclass'),
        pivota_feature_com_sobrev('Cabin'))

Todas estas features aparentam impactar nas chances de sobrevivência; devem ser preenchidas e convertidas para variáveis numéricas.

In [ ]:
# Convertendo a feature "sex" de nominal para binária
all_data['Sex'].replace(['female','male'], [0, 1],inplace=True) 

# Preenchendo 2 valores nulos com o porto de embarque mais comum
all_data['Embarked'].fillna('S', inplace=True)

# Convertendo a feature "Embarked" de nominal para discreta
all_data['Embarked'].replace(['S', 'Q', 'C'], [0, 1, 2],inplace=True)

In [ ]:
display(pivota_feature_com_sobrev('Title').T, 
        pivota_feature_com_sobrev('Parch').T, 
        pivota_feature_com_sobrev('SibSp').T)

Nota-se que grande parte dos indivídios que tiveram companhia para a viagem ou Títulos raros tiveram mais de 50% de chance de sobreviver. Entretanto muitos desses elementos não possuem um número de indivíduos alto o suficiente para serem representativos do todo, transformar esta feature em uma variável ordinal pode incorrer em problemas de amostragem para determiandos valores, fazendo mais sentido criar as seguintes variáveis binárias: 

- "Family": 0 para indivíduos sozinhos e 1 para acompanhados.
- "Title": 1 para indivíduos com títulos raros e 0 para títulos comuns



In [ ]:
# Transformando a feature "Title"
all_data["Title"] = all_data['Title'].replace(['Mrs', 'Miss', 'Mr'], 0)
all_data.loc[all_data['Title'] != 0, 'Title'] = 1

# Criando a feature "Family"
all_data['Family'] =  all_data["Parch"] + all_data["SibSp"]
all_data.loc[all_data['Family'] > 0, 'Family'] = 1

# Descartando
all_data.drop(['SibSp', 'Parch'], axis=1, inplace=True)

<font size="5">**Matriz de correlações**</font>

In [ ]:
# Split
train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):]

train_df.corr().style.background_gradient(cmap='Blues').set_precision(2)

A correlação entre a idade e a sobrevivência é baixa (-0,07), entretando, se analizamos as curvas de kde encontramos faixas estárias mais propensas a sobreviverem. Isso sugere que a feature é relevante para o modelo e deve ser completada.

In [ ]:
kde_age= sns.FacetGrid(train_df, col='Pclass', row='Sex', hue='Survived')
kde_age.add_legend().set(xlim=(0, 100))
kde_age = kde_age.map(sns.kdeplot, 'Age', shade=True)

Uma possibilidade para completar as informações de idade é preenche-las com a mediana das idades, mas este valor pode variar em função de diferentes grupos de pessoas. Vale checar as features "Family" e "Pclass", que são bem correlacionadas com "Age":

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Idade em função da classe e se o passageiro viajou com a familia')
sns.violinplot(x = 'Pclass', y = 'Age', hue = 'Family', data = all_data, split = True, inner="quartile")
plt.show()

**<font size="5">Preenchendo missing values</font>**

Assim como esperado, diferentes grupos possuem diferentes distribuições de idades, e uma vez que possuimos estas informações, é melhor fazer o preenchimento de forma condicionada:

In [ ]:
matriz_de_medianas = np.zeros((2,3))

for classe in range(1, 4):
    for familia in range(0, 2):
        matriz_de_medianas[familia, classe - 1] = all_data.loc[(all_data['Pclass'] == classe) & (all_data['Family'] == familia)]['Age'].median()
        
        all_data.loc[(np.isnan(all_data['Age'])) 
                     & (all_data['Family'] == familia) 
                     & (all_data['Pclass'] == classe), 'Age'] = matriz_de_medianas[familia, classe - 1]

medianas = pd.DataFrame(matriz_de_medianas, columns=[1, 2, 3]); medianas.index.name = 'Family'; medianas.columns.name = 'Pclass'
medianas

O mesmo é realizado para preencher o único valor nulo da feature "Fare"

In [ ]:
all_data.loc[np.isnan(all_data['Fare'])]

In [ ]:
# Preenchendo o valor nulo com a mediana das tarifas deste grupo de indivíduos
all_data.loc[np.isnan(all_data['Fare'])] = all_data.loc[(all_data['Pclass'] == 3) & (all_data['Sex'] == 1) & (all_data['Family'] == 0)]['Fare'].median()

In [ ]:
# Nossos dados estão finalmente organizados, limpos e transformados:
all_data.head()

**<font size="5">Modelagem e predição</font>**

In [ ]:
# Split
train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):]

X_train = train_df.drop("Survived", axis=1)
y_train = train_df["Survived"]
X_test  = test_df.drop("Survived", axis=1)
X_train.shape, y_train.shape, X_test.shape

In [ ]:
# k-nearest neighbor

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test).astype(int)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test).astype(int)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

In [ ]:
feature_importance_df = pd.DataFrame({'feature': X_train.columns, 
                                      'feature_importance': random_forest.feature_importances_}) \
                            .sort_values('feature_importance', ascending = False) \
                            .reset_index(drop = True)

feature_importance_df.columns = ['feature', 'feature_importance']
sns.barplot(x = 'feature_importance', y = 'feature', data = feature_importance_df, orient = 'h', color = 'royalblue') \
   .set_title('feature importance base', fontsize = 20);

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = Id_test
#get predictions
submission['Survived'] = Y_pred
submission.head(15).T

In [ ]:
submission.to_csv('random_forest_submission.csv', index=False)

**Boa parte dos insights mais relevantes para a resolução do problema não partiram de mim. Tenho muito a agredecer pelo trabalho de vários DSs mais experientes da comunidade que compartilharam suas ideias, principalmente através destes notebooks e links:**

- https://www.kaggle.com/alexisbcook/titanic-tutorial

- https://www.kaggle.com/startupsci/titanic-data-science-solutions

- https://www.kaggle.com/omarelgabry/a-journey-through-titanic

- https://www.kaggle.com/tuckerarrants/titanic-ml-top-10

- https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e